# Verified Integer Mathematics in Transformers - Train the Model

This CoLab defines and trains a Transformer model that performs integer addition, subtraction and multiplication e.g. 133357+182243=+0315600, 123450-345670=-0123230 and 000345*000823=+283935. Each digit is a separate token. For 6 digit questions, the model is given 14 "question" (input) tokens, and must then predict the corresponding 8 "answer" (output) tokens.


This CoLab trains the model, storing the results to the Colab files. Useful models are manually copied to HuggingFace.

## Tips for using the Colab
 * You can run and alter the code in this CoLab notebook yourself in Google CoLab ( https://colab.research.google.com/ ).
 * To run the notebook, in Google CoLab, **you will need to** go to Runtime > Change Runtime Type and select GPU as the hardware accelerator.
 * Some graphs are interactive!
 * Use the table of contents pane in the sidebar to navigate.
 * Collapse irrelevant sections with the dropdown arrows.
 * Search the page using the search in the sidebar, not CTRL+F.

# Part 0: Import libraries
Imports standard libraries. Do not read.

Imports "verified_transformer" public library as "qt". This library is specific to this CoLab's "QuantaTool" approach to transformer analysis. Refer https://github.com/PhilipQuirke/verified_transformers/blob/main/README.md for more detail.

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = True
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")

    !pip install kaleido
    !pip install transformer_lens
    !pip install circuitsvis
    !pip install torchtyping
    !pip install transformers

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 37.8 MB/s 

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import kaleido
import plotly.io as pio

if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

import plotly.express as px
import plotly.graph_objects as go

Using renderer: colab


In [3]:
pio.templates['plotly'].layout.xaxis.title.font.size = 20
pio.templates['plotly'].layout.yaxis.title.font.size = 20
pio.templates['plotly'].layout.title.font.size = 30

In [4]:
import json
import requests
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import tqdm.auto as tqdm
import random
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import circuitsvis as cv
import math

In [5]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [6]:
!pip install --upgrade git+https://github.com/PhilipQuirke/verified_transformers.git
import QuantaTools as qt

  Cloning https://github.com/PhilipQuirke/verified_transformers.git to /tmp/pip-req-build-1q1tbwkx
  Running command git clone --filter=blob:none --quiet https://github.com/PhilipQuirke/verified_transformers.git /tmp/pip-req-build-1q1tbwkx
  Resolved https://github.com/PhilipQuirke/verified_transformers.git to commit b067d2ba27101032344397b5748507b8024b97c8
  Preparing metadata (setup.py) ... done
  Created wheel for QuantaTools: filename=QuantaTools-0.1-py3-none-any.whl size=60387 sha256=652ee77e36c82e5da836eb9e2fda5093410e30f28c8b2d18e322f1bf288b2167
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ajws2of/wheels/08/5c/b6/f02e46eb3b254e4572204214b92209ffbe2d0d4a5a61d3adb1
Successfully built QuantaTools


# Part 1A: Configuration


This CoLab can be configured to:
- Train a model using traditional approach. For example, add_d6_l2_h3_t15K_s372001.pth is trained from scratch using 100% addition questions to give an "Addition" model with a very low loss (9e-9).
- Train a "mixed" model to do two tasks by inserting a "known good" model into the untrained composite model. For example, initialising  ins1_mix_d6_l3_h4_t20K_s372001.pth with  add_d6_l2_h3_dm510_dh170_t15K_s372001, and then training it on 80% subtraction questions and 20% addition questions.

If we are inserting (initialising) the model with existing addition model weightings they are loaded from HuggingFace.

In [68]:
# Main configuration class for main model creation and training.
# Derived from MathsConfig > AlgoConfig > UsefulConfig > ModelConfig
class ColabConfig(qt.MathsConfig):

  def __init__(self):
    super().__init__()

    self.main_model = None

    self.insert_late: bool = False
    self.insert_n_layers: int = 2
    self.insert_n_heads: int = 3
    self.insert_training_seed: int = 372001
    self.insert_n_training_steps: int = 15000

    # Save graphs to CoLab temp files as PDF and HTML. Can manually export files for re-use in papers.
    self.save_graph_to_file: bool = True

    # Batch size for training
    self.batch_size = 64


  def to_dict(self):
    return {
      "n_layers": self.n_layers,
      "n_heads": self.n_heads,
      "d_vocab": self.d_vocab,
      "d_mlp": self.d_mlp(),
      "d_head": self.d_head,
      "training_seed": self.training_seed,
      "n_digits": self.n_digits,
      "n_ctx": self.n_ctx(),
      "act_fn": self.act_fn,
      "batch_size": self.batch_size,
      "n_training_steps": self.n_training_steps,
      "lr": self.lr,
      "weight_decay": self.weight_decay,
      "perc_mult": self.perc_mult,
      "perc_sub": self.perc_sub,
      "insert_late": self.insert_late,
      "insert_mode": self.insert_mode,
      "insert_n_layers": self.insert_n_layers,
      "insert_n_heads": self.insert_n_heads,
      "insert_training_seed": self.insert_training_seed,
      "insert_n_training_steps": self.insert_n_training_steps,
    }

In [69]:
# Singleton QuantaTool "main" configuration class. MathsConfig is derived from the chain AlgoConfig > UsefulConfig > ModelConfig
cfg = ColabConfig()


# Which model do we want to analyze? Uncomment one line:

#cfg.model_name = "" # Use default configuration specified in cfg

# 5-digit and 6-digit digit Addition models
#cfg.model_name = "add_d5_l1_h3_t15K_s372001"  # Inaccurate as only has one layer. Can predict S0, S1 and S2 complexity questions.
#cfg.model_name = "add_d5_l2_h3_t15K_s372001"  # AvgFinalLoss=1.6e-08. Six 9s accuracy
#cfg.model_name = "add_d6_l2_h3_t15K_s372001"  # AvgFinalLoss=1.7e-08. Six 9s accuracy. MAIN FOCUS
#cfg.model_name = "add_d6_l2_h3_t20K_s173289"  # AvgFinalLoss=1.5e-08. Six 9s accuracy
#cfg.model_name = "add_d6_l2_h3_t20K_s572091"  # AvgFinalLoss=7e-09. Six 9s accuracy
#cfg.model_name = "add_d10_l2_h3_t40K_s572091"  # AvgFinalLoss=8e-09??. Six 9s accuracy. Fails on 0000000555+0000000445=+00000001000 ModelAnswer: +00000000900

# 6-digit Subtraction model
#cfg.model_name = "sub_d6_l2_h3_t30K_s372001"  # AvgFinalLoss=5.8e-06. Accuracy??

# 6-digit Mixed (addition and subtraction) model
#cfg.model_name = "mix_d6_l3_h4_t40K_s372001"  # AvgFinalLoss=5e-09. Accuracy??

# "ins1" 6-digit Mixed models initialized with 6-digit addition model
#cfg.model_name = "ins1_mix_d6_l3_h4_t40K_s372001"  # AvgFinalLoss=8e-09. Six 9s accuracy for Add and Sub. MAIN FOCUS
#cfg.model_name = "ins1_mix_d6_l3_h4_t40K_s173289"  # AvgFinalLoss=1.6e-08. Accuracy??
#cfg.model_name = "ins1_mix_d6_l3_h4_t50K_s572091"  # AvgFinalLoss=2.9e-08. Accuracy??
#cfg.model_name = "ins1_mix_d6_l3_h3_t40K_s572091"  # AvgFinalLoss=1.8e-08. Accuracy??

# "ins2" 6-digit Mixed model initialized with 6-digit addition model. Reset useful heads every 100 epochs.
#cfg.model_name = "ins2_mix_d6_l4_h4_t40K_s372001"  # AvgFinalLoss=7e-09. Accuracy??

# "ins3" 6-digit Mixed model initialized with 6-digit addition model. Reset useful heads & MLPs every 100 epochs.
cfg.model_name = "ins3_mix_d6_l4_h3_t40K_s372001"  # AvgFinalLoss=3e-04. Accuracy??

# Part 1B: Configuration: Input and Output file names

In [70]:
# Needed when user changes model_name and reruns this Colab a second time
cfg.reset_useful()
cfg.reset_algo()
cfg.initialize_maths_token_positions()

if cfg.model_name != "":
  # Update cfg member data n_digits, n_layers, n_heads, n_training_steps, training_seed from model_name
  cfg.parse_model_name()

  # Addition model
  cfg.perc_sub = 0
  if cfg.model_name.startswith("sub_") :
    # Subtraction model
    cfg.perc_sub = 100
  elif cfg.model_name.startswith("mix") :
    # Mixed (addition and subtraction) model
    cfg.perc_sub = 66 # Train on 66% subtraction and 33% addition question batches
  elif cfg.model_name.startswith("ins") :
    # Mixed model initialised with an addition model (using insert mode 1, 2 or 3)
    cfg.perc_sub = 80 # Train on 80% subtraction and 20% addition question batches

  # We train multiple versions of some models, inserting different addition models.
  cfg.insert_training_seed = cfg.training_seed

  if cfg.model_name.startswith("ins1_mix_d6_l3") :
    if cfg.training_seed == 372001:
      # Mixed model initialised with add_d6_l2_h3_t15K.pth.
      cfg.insert_n_training_steps = 15000
    else:
      # Mixed model initialised with add_d6_l2_h3_t20K.pth.
      cfg.insert_n_training_steps = 20000

In [71]:
main_fname = cfg.file_config_prefix()
main_fname_pth = main_fname + '.pth'
main_fname_json = main_fname + '_train.json'


def print_config():
  print("%Add=", cfg.perc_add(), "%Sub=", cfg.perc_sub, "%Mult=", cfg.perc_mult, "InsertMode=", cfg.insert_mode, "File=", main_fname)

print_config()
print("weight_decay=", cfg.weight_decay, "lr=", cfg.lr, "batch_size=", cfg.batch_size)
print('Main model will save to Colab temporary file', main_fname_pth)
print('Main model config etc will save to Colab temporary file', main_fname_json)

%Add= 20 %Sub= 80 %Mult= 0 InsertMode= 3 File= ins3_mix_d6_l4_h3_t40K_s372001
weight_decay= 0.1 lr= 8e-05 batch_size= 64
Main model will save to Colab temporary file ins3_mix_d6_l4_h3_t40K_s372001.pth
Main model config etc will save to Colab temporary file ins3_mix_d6_l4_h3_t40K_s372001_train.json


In [72]:
# Singleton QuantaTool "ablation intervention" configuration class
acfg = qt.acfg
acfg.reset_ablate()

# Part 3A: Set Up: Vocabulary / Embedding / Unembedding

In [73]:
qt.set_maths_vocabulary(cfg)
qt.set_maths_question_meanings(cfg)
print(cfg.token_position_meanings)

['D5', 'D4', 'D3', 'D2', 'D1', 'D0', 'OPR', "D'5", "D'4", "D'3", "D'2", "D'1", "D'0", '=', 'A7', 'A6', 'A5', 'A4', 'A3', 'A2', 'A1', 'A0']


# Part 3B: Create main_model
This section defines the token embedding / unembedding and creates the model.

In [74]:
# Transformer creation

# Structure is documented at https://neelnanda-io.github.io/TransformerLens/transformer_lens.html#transformer_lens.HookedTransformerConfig.HookedTransformerConfig
ht_cfg = HookedTransformerConfig(
    n_layers = cfg.n_layers,
    n_heads = cfg.n_heads,
    d_model = cfg.d_model,
    d_head = cfg.d_head,
    d_mlp = cfg.d_mlp(),
    act_fn = cfg.act_fn,
    normalization_type = 'LN',
    d_vocab = cfg.d_vocab,
    d_vocab_out = cfg.d_vocab,
    n_ctx = cfg.n_ctx(),
    init_weights = True,
    device = "cuda",
    seed = cfg.training_seed,
)

cfg.main_model = HookedTransformer(ht_cfg)

optimizer = optim.AdamW(cfg.main_model.parameters(),
                        lr = cfg.lr,
                        weight_decay = cfg.weight_decay,
                        betas = (0.9, 0.98))

max_iter = cfg.n_training_steps
warmup_iter = max_iter // 5
scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.01, total_iters=int(warmup_iter))
scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(np.ceil((max_iter-warmup_iter))))
scheduler  = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[int(warmup_iter)])

# Part 4: Data Generator
This section defines the training/testing data generator.


In [75]:
# Define "iterator" maths "questions" data generator function. Invoked using next().
ds = qt.maths_data_generator( cfg )

In [76]:
# Test data generator
tokens = next(ds)
print(tokens[:3,:])

tensor([[ 4,  9,  7,  0,  1,  8, 11,  8,  8,  1,  0,  7,  4, 12, 11,  0,  3,  8,
          4,  0,  5,  6],
        [ 9,  2,  8,  9,  9,  4, 11,  8,  3,  3,  7,  9,  1, 12, 10,  0,  0,  9,
          5,  2,  0,  3],
        [ 5,  8,  9,  8,  9,  3, 11,  9,  2,  0,  6,  6,  3, 12, 11,  0,  3,  3,
          0,  7,  7,  0]], device='cuda:0')


# Part 5: Read insert_model from HuggingFace (optional)

If we are initialising the untrained model with an existing model,
then we load the existing model from HuggingFace.
We load both the model weights and a json file stating which nodes in the model are actually doing useful calculations.

In [77]:
insert_base_name = ""
insert_weights_fname = ""
insert_nodes_fname = ""

In [78]:
# Read insert_model weights from HuggingFace
if cfg.insert_mode >= 1:
  train_str = str(cfg.insert_n_training_steps//1000) + "K"
  insert_base_name = 'add_d{}_l{}_h{}_t{}_s{}'.format(cfg.n_digits, cfg.insert_n_layers, cfg.insert_n_heads, train_str, cfg.insert_training_seed)

  insert_weights_fname = insert_base_name + ".pth"

  ht_cfg = HookedTransformerConfig(
      n_layers = cfg.insert_n_layers,
      n_heads = cfg.insert_n_heads,
      d_model = cfg.d_model, # Assume constant
      d_head = cfg.d_head, # Assume constant
      d_mlp = cfg.d_mlp(), # Assume constant
      act_fn = cfg.act_fn, # Assume constant
      normalization_type = 'LN',
      d_vocab = cfg.d_vocab, # Assume constant
      d_vocab_out = cfg.d_vocab, # Assume constant
      n_ctx = cfg.n_ctx(), # Assume constant
      init_weights = True, # Assume constant
      device = "cuda",
      seed = cfg.insert_training_seed,
  )

  insert_model = HookedTransformer(ht_cfg)

  print('Loading insertion model from', insert_weights_fname)
  insert_model.load_state_dict(utils.download_file_from_hf(repo_name="PhilipQuirke/VerifiedArithmetic", file_name=insert_weights_fname, force_is_torch=True))
  insert_model.eval()

  print("Loaded insert model", insert_weights_fname)

Loading insertion model from add_d6_l2_h3_t15K_s372001.pth
Loaded insert model add_d6_l2_h3_t15K_s372001.pth


In [79]:
if cfg.insert_mode >= 1:
  # Read insert_model useful node information from HuggingFace
  huggingface_directory_url = 'https://huggingface.co/PhilipQuirke/VerifiedArithmetic/raw/main/'
  insert_nodes_fname = huggingface_directory_url + insert_base_name + '_behavior.json'
  print(insert_nodes_fname)

  # Download the file
  response = requests.get(insert_nodes_fname)

  # Ensure the request was successful
  if response.status_code == 200:
      # Load the JSON data
      json_data = json.loads(response.content.decode('utf-8'))

      useful_cells = [qt.UsefulNode.from_dict(item) for item in json_data]

      print( "Loaded:", len(useful_cells), "Sample:", useful_cells[0].tags)
  else:
      print( "Failed to download the file:", response.status_code)

https://huggingface.co/PhilipQuirke/VerifiedArithmetic/raw/main/add_d6_l2_h3_t15K_s372001_behavior.json
Loaded: 48 Sample: ['Fail%:12', 'Impact:A654', 'Math.Add:S1234', 'Attn:P3=51', 'Attn:P10=48', 'Math.Add:A2.SP']


# Part 6B: Transfer all of insert_model into main_model (optional)



In [80]:
# Transfer all attention heads weights from the small to the main model, updating the right-most small.n_heads of main_model
def transfer_all_heads(small_model, small_cfg, start_layer, end_layer, large_model):
  for small_layer_no, large_layer_no in enumerate(range(start_layer, end_layer+1)):
    large_model.blocks[large_layer_no].attn.W_Q.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.W_Q.clone().data
    large_model.blocks[large_layer_no].attn.W_K.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.W_K.clone().data
    large_model.blocks[large_layer_no].attn.W_V.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.W_V.clone().data

    large_model.blocks[large_layer_no].attn.b_Q.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.b_Q.clone().data
    large_model.blocks[large_layer_no].attn.b_K.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.b_K.clone().data
    large_model.blocks[large_layer_no].attn.b_V.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.b_V.clone().data

In [81]:
# Transfer all MLP layer weights from the small to the main model, updating the right-most small.d_mlp of main_model
def transfer_all_mlps(small_model, small_cfg, start_layer, end_layer, large_model):
  for small_layer_no, large_layer_no in enumerate(range(start_layer, end_layer+1)):
    large_model.blocks[large_layer_no].mlp.W_in.data[:, :small_cfg["d_mlp"]] = small_model.blocks[small_layer_no].mlp.W_in.clone().data
    large_model.blocks[large_layer_no].mlp.b_in.data[:small_cfg["d_mlp"]] = small_model.blocks[small_layer_no].mlp.b_in.clone().data
    large_model.blocks[large_layer_no].mlp.W_out.data[:small_cfg["d_mlp"],] = small_model.blocks[small_layer_no].mlp.W_out.clone().data
    large_model.blocks[large_layer_no].mlp.b_out.data = small_model.blocks[small_layer_no].mlp.b_out.clone().data

In [82]:
def transfer_all_ln(small_model, start_layer, end_layer, large_model):
  for small_layer_no, large_layer_no in enumerate(range(start_layer, end_layer+1)):
    large_model.blocks[large_layer_no].ln1.w.data = small_model.blocks[small_layer_no].ln1.w.clone().data
    large_model.blocks[large_layer_no].ln1.b.data = small_model.blocks[small_layer_no].ln1.b.clone().data

  large_model.ln_final.w.data = small_model.ln_final.w.clone().data

In [83]:
def transfer_all_embeds(small_model, large_model):
  large_model.embed.W_E.data = small_model.embed.W_E.clone().data
  large_model.pos_embed.W_pos.data = small_model.pos_embed.W_pos.clone().data
  large_model.unembed.W_U.data = small_model.unembed.W_U.clone().data

In [84]:
small_cfg = {}
large_cfg = {}

# Insert the small model weights into the large model
def transfer_full_model(small_model, large_model, start_layer, end_layer, transfer_ln=True, transfer_embeds=True):
  """Args:
  small_model: The model to transfer weights from
  large_model: The model to transfer weights to
  start_layer: The first layer to transfer weights to
  end_layer: The last layer to transfer weights to (Note that this is end-inclusive!)
  """
  global small_cfg
  global large_cfg

  small_cfg = {k: v for k,v in small_model.cfg.__dict__.items() if k in ["d_head", "d_mlp", "d_model", "n_heads", "n_layers"]}
  large_cfg = {k: v for k,v in large_model.cfg.__dict__.items() if k in ["d_head", "d_mlp", "d_model", "n_heads", "n_layers"]}

  # Sanity checks for large model > small model
  assert small_cfg["d_model"] == large_cfg["d_model"]
  assert small_cfg["d_head"] == large_cfg["d_head"]
  assert small_cfg["n_layers"] <= large_cfg["n_layers"]
  assert small_cfg["n_heads"] <= large_cfg["n_heads"]
  assert small_cfg["d_mlp"] <= large_cfg["d_mlp"]

  assert 0 <= start_layer < end_layer <= large_cfg["n_layers"] # Make sure start_layer and end_layer are valid
  assert end_layer - start_layer + 1 == small_cfg["n_layers"] # Make sure the number of layers to transfer is correct

  transfer_all_heads(small_model, small_cfg, start_layer, end_layer, large_model)
  transfer_all_mlps(small_model, small_cfg, start_layer, end_layer, large_model)
  if transfer_ln:
    transfer_all_ln(small_model, start_layer, end_layer, large_model)
  if transfer_embeds:
    transfer_all_embeds(small_model, large_model)

In [85]:
def insert_existing_model( first_time ):
  if cfg.insert_mode >= 1 :
    # Is the destination the first few or last few layers of the main_model?
    start_layer = cfg.n_layers - cfg.insert_n_layers if cfg.insert_late else 0
    end_layer = start_layer + cfg.insert_n_layers-1

    if first_time:
      print( "Inserting trained small_model", insert_weights_fname)
      print( "into larger, untrained main_model", main_fname)
      print( "destination layers:", start_layer, end_layer)

    transfer_full_model(insert_model, cfg.main_model, start_layer, end_layer, first_time, first_time)


insert_existing_model( True )

Inserting trained small_model add_d6_l2_h3_t15K_s372001.pth
into larger, untrained main_model ins3_mix_d6_l4_h3_t40K_s372001
destination layers: 0 1


# Part 6C: Transfer useful heads of insert_model into main_model (optional)

Transfer just the useful attention heads from insert_model into main_model.

In [86]:
# Transfer one attention head's weights from the small to the main model.
# The right-most small.n_heads of main_model are updated
def transfer_one_head(small_model, small_layer_no, small_head_no, large_model, start_layer):
  large_layer_no = start_layer + small_layer_no
  large_head_no = large_cfg["n_heads"] - small_cfg["n_heads"] + small_head_no

  large_model.blocks[large_layer_no].attn.W_Q.data[large_head_no] = small_model.blocks[small_layer_no].attn.W_Q.clone().data[small_head_no]
  large_model.blocks[large_layer_no].attn.W_K.data[large_head_no] = small_model.blocks[small_layer_no].attn.W_K.clone().data[small_head_no]
  large_model.blocks[large_layer_no].attn.W_V.data[large_head_no] = small_model.blocks[small_layer_no].attn.W_V.clone().data[small_head_no]

  large_model.blocks[large_layer_no].attn.b_Q.data[large_head_no] = small_model.blocks[small_layer_no].attn.b_Q.clone().data[small_head_no]
  large_model.blocks[large_layer_no].attn.b_K.data[large_head_no] = small_model.blocks[small_layer_no].attn.b_K.clone().data[small_head_no]
  large_model.blocks[large_layer_no].attn.b_V.data[large_head_no] = small_model.blocks[small_layer_no].attn.b_V.clone().data[small_head_no]

In [87]:
def transfer_useful_heads(small_model, large_model):
  if cfg.insert_mode >= 2 and len(useful_cells) > 0:
    # Is the destination the first few or last few layers of the main_model?
    start_layer = cfg.n_layers - cfg.insert_n_layers if cfg.insert_late else 0

    # TODO: Somewhat inefficent loop as a given head may exist in the list multiple times (with different use_cell.position and use_cell.tags values)
    for use_cell in useful_cells:
      if use_cell.is_head:
        transfer_one_head(small_model, use_cell.layer, use_cell.num, large_model, start_layer)

# Part 7: Train add/sub/mix main_model with Infinite Data
Train main_model for n_training_steps, storing train_losses per epoch.

Each training step (of n_training_steps) new training data (a batch of batch_size tokens) is generated and the model is trained and loss calculated on it. No separate "testing" data is need   ed, as the training data is unique each step. Memorisation of past training data by the model (if any) is minimally beneficial. For 6 digit addition or subtraction there are 1000 billion possible questions.

In [ ]:
print_config()

# Train the model
train_losses_list = []
per_token_train_losses_list = []

for epoch in tqdm.tqdm(range(cfg.n_training_steps)):

  tokens = next(ds)
  logits = cfg.main_model(tokens)

  per_token_train_losses_raw, _ = qt.logits_to_tokens_loss(cfg, logits, tokens)
  per_token_train_losses = qt.loss_fn(per_token_train_losses_raw)
  per_token_train_losses_list.append(utils.to_numpy(per_token_train_losses))

  train_loss = per_token_train_losses.mean()
  train_loss.backward()
  train_losses_list.append(train_loss.item())

  optimizer.step()
  scheduler.step()
  optimizer.zero_grad()

  if epoch % 100 == 0:
    print(epoch, train_loss.item())
    if cfg.insert_mode == 2:
      # Freeze the useful attention heads from insert_model
      transfer_useful_heads(insert_model, cfg.main_model)
    if cfg.insert_mode == 3:
      # Freeze the useful attention heads and MLP layers from insert_model
      insert_existing_model( False )

print(epoch, train_loss.item())

%Add= 20 %Sub= 80 %Mult= 0 InsertMode= 3 File= ins3_mix_d6_l4_h3_t40K_s372001


  0%|          | 0/40000 [00:00<?, ?it/s]

0 11.289879751878278


In [ ]:
final_training_loss = round((train_losses_list[-5]+train_losses_list[-4]+train_losses_list[-3]+train_losses_list[-2]+train_losses_list[-1])/5,9)

print( "AvgFinalLoss", final_training_loss)
print( "FinalLoss", train_losses_list[-1])

In [ ]:
# These temporary Colab files can be manually downloaded from the Colab "Files" tab (at left).
# The download can be manually loaded into HuggingFace so the "VerifiedArithmeticAnalyse" Colab can access it.

print("Saving main model to temporary Colab file", main_fname_pth)
torch.save(cfg.main_model.state_dict(), main_fname_pth)

In [ ]:
extra_data = {
    "Config": cfg.to_dict(),
    "AvgFinalLoss": final_training_loss,
    "FinalLoss": train_losses_list[-1],
    "TrainingLoss": train_losses_list
}

print( "Saving main model config etc to temporary Colab file:", main_fname_json)
save_cfg = cfg.to_dict()
with open(main_fname_json, 'w') as file:
    json.dump(extra_data, file)

# Part 9: Line Graphs

This section analyses the training loss by graphing it at a high level.

The loss curve for all digits show visible inflection points (bumps), but is too high level to help understand the algorithm.

When this graph is decomposed into 'per digit' graphs, the interesting distinct 'per digit' curves appear, showing each digit is being refined semi-independently, with the model algorithm refining each digit separately.

In [ ]:
epochs_to_graph=1500

In [ ]:
def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

# Helper function to plot multiple lines
def lines(raw_lines_list, x=None, mode='lines', labels=None, xaxis='Epoch', yaxis='Loss', title = '', log_y=False, hover=None, all_epochs=True, **kwargs):
    global epochs_to_graph

    lines_list = raw_lines_list if all_epochs==False else [row[:epochs_to_graph] for row in raw_lines_list]
    log_suffix = '' if log_y==False else ' (Log)'
    epoch_suffix = '' if all_epochs==False else ' (' + str(epochs_to_graph) + ' Epochs)'
    full_title = title + log_suffix + epoch_suffix

    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    if cfg.save_graph_to_file :
      fig = go.Figure(layout={})
      print(full_title)
    else:
      fig = go.Figure(layout={'title':full_title})

    fig.update_xaxes(
        title=xaxis,
        showgrid=False)
    fig.update_yaxes(
        title=yaxis + log_suffix,
        showgrid=False)

    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))

    if log_y:
        fig.update_layout(yaxis_type="log")
    else:
      # Calculate the max y-value rounded up to the nearest integer
      y_max = 1
      for k in range(len(lines_list)):
          y_max = max(y_max, math.ceil(max(lines_list[k])) )
      y_max = 3 # manual override if necessary

      # Update layout to set the y-axis min to 0 and max to the calculated y_max
      fig.update_layout(
          yaxis=dict(range=[0, y_max])
      )

      # Update x-axis ticks
      x_ticks = x[0::100]  # Start from index 0 and pick every 100th element
      x_ticks = x_ticks[1:] # Exclude the first tick (0)
      fig.update_xaxes(
          tickmode='array',
          tickvals=x_ticks,
          ticktext=[str(tick) for tick in x_ticks]
      )

    if cfg.save_graph_to_file:
        # fig.update_layout(margin=dict(l=10, r=10, t=10, b=10),width=1200,height=300)
        # Update layout for legend positioning inside the graph
        fig.update_layout(
            margin=dict(l=10, r=10, t=10, b=10),
            width=1200,height=300,
            legend=dict(
                x=0.92,  # Adjust this value to move the legend left or right
                y=0.99,  # Adjust this value to move the legend up or down
                traceorder="normal",
                font=dict(
                    family="sans-serif",
                    size=12,
                    color="black"
                ),
                bgcolor="White",  # Adjust background color for visibility
                bordercolor="Black",
                borderwidth=2
            ))

    fig.show(bbox_inches="tight")

    if cfg.save_graph_to_file:
        filename = full_title.replace(" ", "").replace("(", "").replace(")", "").replace("&", "").replace(",", "").replace("%", "")   +'.pdf'
        pio.write_image(fig, filename)

In [ ]:
title_suffix = 'Digit Loss Curves ' + main_fname
per_token_losses = np.stack(per_token_train_losses_list, axis=0)

line(train_losses_list,
    title=title_suffix)

answer_digits = cfg.n_digits + 1
all_epochs = True;
for i in range(2):
  lines([per_token_losses[:, i] for i in range(answer_digits)]+[train_losses_list],
        labels = [f'A{cfg.n_digits-i}' for i in range(answer_digits)]+['All'],
        title='Per digit'+title_suffix,
        all_epochs=all_epochs)

  lines([per_token_losses[:, i] for i in range(answer_digits)]+[train_losses_list],
        labels = [f'A{cfg.n_digits-i}' for i in range(answer_digits)]+['All'],
        title='Per digit'+title_suffix,
        all_epochs=all_epochs,
        log_y=True)

  all_epochs = False

for i in range(answer_digits):
  print('Final Loss for A' + str(cfg.n_digits-i) + ' is ', per_token_losses[-1, i])

# Part 10: Questions Set Up

Create sets of sample questions (by task) to ask the model to predict

In [ ]:
def make_varied_questions():
  q0 = next(ds)
  q1 = next(ds)
  q2 = next(ds)
  q3 = next(ds)

  questions = torch.vstack((q0.cuda(), q1.cuda(), q2.cuda(), q3.cuda()))

  return questions

In [ ]:
varied_questions = make_varied_questions()
num_varied_questions = varied_questions.shape[0]

qt.a_set_ablate_hooks(cfg)
qt.a_calc_mean_values(cfg, varied_questions)

cfg.main_model.reset_hooks()
cfg.main_model.set_use_attn_result(True)
sample_logits, sample_cache = cfg.main_model.run_with_cache(varied_questions.cuda())

# Part 11: Attention Patterns
Attention patterns show which token(s) the model's attention heads are paying attention to in each token position of the prediction calculation.

For the default CoLab set up, the  model has 3 attention heads, and performs 5 digit addition. The attention pattern is 18 by 18 squares (as 54321+77779=132100 is 18 tokens). Time proceeds vertically downwards, with one additional token being revealed horizontally at each token position, giving the overall triangle shape. This visualisation provided insights. After the question is fully revealed (at token position 11), each head starts attending to pairs of question digits from left to right (i.e. high-value digits before lower-value digits) giving the “double staircase" shape. The three heads attend to a given digit pair in three different token position, giving a time ordering of heads.

In [ ]:
def show_token_attention_patterns(index, layer, token_at_index, use_case):

  the_tokens = [str(token) for token in token_at_index.tolist()]
  if layer == 0:
    tokens_str = qt.tokens_to_string(cfg, token_at_index)
    print("Attention patterns for", tokens_str)

  attention_pattern=sample_cache["pattern", layer, "attn"][index]
  display(cv.attention.attention_patterns(
      tokens=the_tokens,
      attention=attention_pattern,
      #attention_head_names=[f"L{layer}H{i}" for i in range(cfg.n_heads)],
  ))


sample_size = 3

# Show attention patterns for some randomly chosen tokens
for i in range(sample_size):
  for layer in range(cfg.n_layers):
    show_token_attention_patterns(i, layer, tokens[i], "Misc")


In [ ]:
if cfg.save_graph_to_file:

  tokens_str = []
  for i in range(cfg.n_heads):
    one_token_str = []
    for j in tokens[i]:
      one_token_str.append(str(utils.to_numpy(j)))
    tokens_str.append(one_token_str)

  # Refer https://github.com/callummcdougall/CircuitsVis/blob/main/python/circuitsvis/circuitsvis_demo.ipynb

  # html_object = cv.attention.from_cache(
  #    cache = sample_cache,
  #    tokens = tokens_str, # list of list of strings
  #    return_mode = "html",
  #)

  # Create a CoLab file containing the attention pattern(s) in HTML
  #filename = "AttentionPattern" + str(cfg.n_digits) + "Digits" + str(cfg.n_heads) + "Heads.html"
  #with open(filename, "w") as f:
  #    f.write(html_object.data)

  # Manually download the CoLab "html" file and open in your local browser.
  # Install and use the Edge extension "FireShot" to save a portion of the HTML page as a PDF